<a href="https://colab.research.google.com/github/danosaku/Classificacao_cardapios_escolares/blob/main/nutricao_cardapio_multisource_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Classificacao de componentes de avaliação do método de avaliação IQCAE-R usando as descrições dos cardápios escolares como sendo um problema multi-rótulos.

# Instalando e Importando Bibliotecas


### Pandas e Numpy

In [ ]:
import pandas as pd
import numpy as np
!pip install scikit-multilearn
from skmultilearn.model_selection import iterative_train_test_split


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### NLTK
* Apoio ao pré-processamento de textos (tokenização, stopwords, radicalização)

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('rslp')
from nltk.tokenize import word_tokenize
from nltk.stem.porter import *
from nltk.stem import RSLPStemmer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


Implementação de uma MLP em KERAS

In [ ]:
from keras.layers import Dense, Dropout, Conv1D,  Activation, BatchNormalization, Add
from keras.layers import Input, Flatten, Concatenate
from keras.models import Model, load_model
from tensorflow.keras import layers
from keras.layers.pooling import GlobalMaxPooling2D, GlobalAveragePooling1D, MaxPooling2D, AveragePooling2D
import keras
import gc
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import accuracy_score, confusion_matrix, cohen_kappa_score
import tensorflow as tf
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath='/content/tmp.h5',
    save_weights_only=True,
    monitor='loss',
    mode='min',
    save_best_only=True)

In [ ]:
class mlp(BaseEstimator, TransformerMixin):
    """
    Multi-Layer Perceptron for Pipeline
    """

    def __init__(self, input_size=100, input_size2=100, input_size3 = 100, input_size4 = 100, nb_filter = 16, lr = 1e-3, epochs = 1, batch_size = 8):
        self.size = input_size
        self.size2 = input_size2
        self.size3 = input_size3
        self.size4 = input_size4
        self.nb_filter = nb_filter
        self.lr = lr
        self.epochs = epochs
        self.model = None
        self.batch_size = batch_size

    def fit(self, X, y=None):

        self.model.fit(X, y,
                   batch_size=self.batch_size,
                   epochs=self.epochs,
                   verbose=1,
                   callbacks = [model_checkpoint_callback ]
                   )

        self.model.load_weights('/content/tmp.h5')
        !rm '/content/tmp.h5'
        return self

    def predict(self, X):
        return self.model.predict(X, batch_size = self.batch_size, verbose=0)

    def mlp(self, nclasses):
     eps = 1.1e-5
     input_vector = Input(shape=self.size)
     input_vector2 = Input(shape=self.size2)
     input_vector3 = Input(shape=self.size3)
     input_vector4 = Input(shape=self.size4)
     x = Dense(2*self.nb_filter, name='conv1', use_bias=True)(input_vector)
     x = BatchNormalization(epsilon=eps)(x)
     x = Activation('relu', name='relu1')(x)
          #x = Dropout(0.5)(x)
     x1 = Dense(self.nb_filter, name='conv2', use_bias=True)(x)
     x1 = BatchNormalization(epsilon=eps)(x1)
     x1 = Activation('relu', name='relu2')(x1)
     #x = Concatenate(axis=1)([x, x1])

     x2 = Dense(2*self.nb_filter, name='conv1_tipo', use_bias=True)(input_vector)
     x2 = BatchNormalization(epsilon=eps)(x2)
     x2 = Activation('relu', name='relu1_tipo')(x2)
     x3 = Dense(self.nb_filter, name='conv2_tipo', use_bias=True)(x2)
     x3 = BatchNormalization(epsilon=eps)(x3)
     x3 = Activation('relu', name='relu2_tipo')(x3)
     x_tipo = Concatenate(axis=1)([x2, x3])
     #x = Dense(1000, name="fc1")(x)
     x_final = Dropout(0.5)(x_tipo)
     x_final = Dense(1, name="fc2_tipo")(x_final)
     x_final = Activation('sigmoid', name='prob1_tipo')(x_final)


     ### Alimentos sociobiodiversidade
     input_est = Concatenate()([input_vector, input_vector4])
     #x_est3 = Dense(2*self.nb_filter, name='conv3_est', use_bias=True)(input_est)
     #x_est3 = BatchNormalization(epsilon=eps)(x_est3)
     #x_est3 = Activation('relu', name='relu3_est')(x_est3)
     #x_est4 = Dense(self.nb_filter, name='conv4_est', use_bias=True)(x_est3)
     #x_est4 = BatchNormalization(epsilon=eps)(x_est4)
     #x_est4 = Activation('relu', name='relu4_est')(x_est4)
     x_est = Dense(2*self.nb_filter, name='conv1_est', use_bias=True)(input_est)
     x_est = BatchNormalization(epsilon=eps)(x_est)
     x_est = Activation('relu', name='relu1_est')(x_est)
     x_est2 = Dense(self.nb_filter, name='conv2_est', use_bias=True)(x_est)
     x_est2 = BatchNormalization(epsilon=eps)(x_est2)
     x_est2 = Activation('relu', name='relu2_est')(x_est2)
     #x_mm = Dense(self.nb_filter, name='conv2_mm', use_bias=True)(x)
     #x_mm = Activation('relu', name='relu2_mm')(x_mm)
     x_est = Concatenate(axis=1)([x_est, x_est2])
    # x_est = Concatenate(axis=1)([input_vector, x_est, x_est2])
     #x = Dense(1000, name="fc1")(x)
     x_est = Dropout(0.5)(x_est)
     x_est = Dense(1, name="fc2_est")(x_est)


     ### Alimentos regionais
     input_reg = Concatenate()([input_vector, input_vector3])
     x_reg = Dense(2*self.nb_filter, name='conv1_reg', use_bias=True)(input_reg)
     x_reg = BatchNormalization(epsilon=eps)(x_reg)
     x_reg = Activation('relu', name='relu1_reg')(x_reg)
     x_reg2 = Dense(self.nb_filter, name='conv2_reg', use_bias=True)(x_reg)
     x_reg2 = BatchNormalization(epsilon=eps)(x_reg2)
     x_reg2 = Activation('relu', name='relu2_reg')(x_reg2)
     #x_rr = Dense(self.nb_filter, name='conv2_rr', use_bias=True)(x)
     #x_rr = Activation('relu', name='relu2_rr')(x_rr)
     #x_reg3 = Dense(2*self.nb_filter, name='conv3_reg', use_bias=True)(input_vector3)
     #x_reg3 = BatchNormalization(epsilon=eps)(x_reg3)
     #x_reg3 = Activation('relu', name='relu3_reg')(x_reg3)
     #x_reg4 = Dense(self.nb_filter, name='conv4_reg', use_bias=True)(x_reg3)
     #x_reg4 = BatchNormalization(epsilon=eps)(x_reg4)
     #x_reg4 = Activation('relu', name='relu4_reg')(x_reg4)
     x_reg = Concatenate(axis=1)([x_reg, x_reg2])
    # x_reg = Concatenate(axis=1)([input_vector, x_reg, x_reg2])
     #x = Dense(1000, name="fc1")(x)
     x_reg = Dropout(0.5)(x_reg)
     x_reg = Dense(1, name="fc2_reg")(x_reg)


     x4 = Dense(2*self.nb_filter, name='conv1_horario', use_bias=True)(input_vector2)
     x4 = BatchNormalization(epsilon=eps)(x4)
     x4 = Activation('relu', name='relu1_horario')(x4)
     x5 = Dense(self.nb_filter, name='conv2_horario', use_bias=True)(x4)
     x5 = BatchNormalization(epsilon=eps)(x5)
     x5 = Activation('relu', name='relu2_horario')(x5)
     #x_14 = Concatenate(axis=1)([x_tipo, x4, x5])
     x_14 = Concatenate(axis=1)([x_tipo, x4, x5])
     x_hor = Dropout(0.5)(x_14)
     x_hor = Dense(1, name="fc2_hor")(x_hor)
     #x_hor = Activation('sigmoid', name='prob1_hor')(x_hor)


     x = Concatenate(axis=1)([x, x1])
     #x = Dense(1000, name="fc1")(x)
     x = Dropout(0.5)(x)
     x = Dense(nclasses, name="fc2")(x)
     x = Concatenate(axis=1)([x, x_reg, x_est, x_hor])
     #x = Dropout(0.5)(x)
     #x = Dense(1, name="fc1")(x)
     x = Activation('sigmoid', name='prob1')(x)
     #x = BatchNormalization()(x)
     #x1 = Dense(1, name="fc2")(x)
     #x1 = Activation('linear', name='prob2')(x1)
     #x1 = BatchNormalization()(x1)
     return Model([input_vector, input_vector2, input_vector3, input_vector4], [x, x_final], name='Recepi_conv')


    def compile(self):
       self.model = self.mlp(11)
       self.model.compile(loss={'prob1': "mean_squared_error",'prob1_tipo': 'binary_crossentropy'}, optimizer="adam", metrics=["mse"])
       for i in range(len(self.model.layers)):
           print(i, self.model.layers[i].name)

    def get_output(self, X):
         idx = [37,38,39,40]
         out = []
         for i in idx:
            intermediate_layer_model = Model(inputs=self.model.input,
                                 outputs=self.model.get_layer(self.model.layers[i].name).output)
            out.append(intermediate_layer_model.predict(X, verbose=2))
         #intermediate_output = intermediate_layer_model.predict(X_valid, verbose=2)
         return out

    def save(self, output):
       self.model.save(output)

    def summary(self):
       self.model.summary()

    def clear_session(self):
         keras.backend.clear_session()
         self.model = None
         gc.collect()


In [ ]:
class cnn(BaseEstimator, TransformerMixin):
    """
    Multi-Layer Perceptron for Pipeline
    """

    def __init__(self, input_size=100, nb_filter = 16, lr = 1e-3, epochs = 1, batch_size = 8, vocab_size = 1000):
        self.size = input_size
        self.nb_filter = nb_filter
        self.lr = lr
        self.epochs = epochs
        self.model = None
        self.batch_size = batch_size
        self.vocab_size = vocab_size

    def fit(self, X, y=None):

        self.model.fit(X, y,
                   batch_size=self.batch_size,
                   epochs=self.epochs,
                   verbose=0,
                   callbacks = [model_checkpoint_callback ]
                   )

        self.model.load_weights('/content/tmp.h5')
        !rm '/content/tmp.h5'
        return self

    def predict(self, X):
        return self.model.predict(X, batch_size = self.batch_size, verbose=0)

    def cnn(self, vocab_size, embedding_dim, maxlen, nclasses):

     input_vector = Input(shape=self.size)
     embedded_sequences = layers.Embedding(self.vocab_size, embedding_dim, input_length=maxlen)(input_vector)
     l_cov1  = layers.Conv1D(317, 3, activation='relu')(embedded_sequences)
     l_pool1 = layers.MaxPooling1D(2)(l_cov1)
     l_cov2  = layers.Conv1D(317, 1, activation='relu')(l_pool1)
     l_cov3  = layers.Conv1D(317, 2, activation='relu')(l_cov2)
     l_pool3 = layers.GlobalMaxPooling1D()(l_cov3)  # global max pooling
     l_bnorm = layers.BatchNormalization()(l_pool3)
     l_dense = layers.Dense(128, activation='relu')(l_pool3)
     preds   = layers.Dense(1, activation='sigmoid',name='preds')(l_dense)
     return Model(input_vector, x, name='Iassist_conv')

    def heat_map(self):
      class_idx = np.argmax(y_pred[0]) #not needed in this case as only two classes
      class_output = self.model.output[:, class_idx]
      last_conv_layer = self.model.get_layer("conv1d_45")
      grads = K.gradients(class_output, last_conv_layer.output)[0]
      pooled_grads = K.mean(grads)
      iterate = K.function([self.model.input], [pooled_grads, last_conv_layer.output[0]])
      pooled_grads_value, conv_layer_output_value = iterate([Xtst])
      heatmap = np.mean(conv_layer_output_value, axis=-1)
      heatmap = np.maximum(heatmap,0)
      heatmap /= np.max(heatmap)#normalise values in the prediction


    def compile(self):
       self.model = self.mlp(1)
       self.model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

    def save(self, output):
       self.model.save(output)

    def summary(self):
       self.model.summary()

    def clear_session(self):
         keras.backend.clear_session()
         self.model = None
         gc.collect()


Pré-processamento dos dados

In [ ]:
def remove_stopwords(text,stop_words):

  ##

                       # adding to stopwords

  # tudo para caixa baixa
  s = str(text).lower()

  tokens = word_tokenize(s)

  # remove stopwords, dígitos, caracteres especiais e pontuações
  v = [word for word in tokens if not word in stop_words and word.isalnum() and not word.isdigit()]

  return v

In [ ]:
def stemming(tokens,stemmer):
  # remove_list = ['aceit', 'acord', 'agradec', 'além', 'algum', 'aniversári', 'ano', 'anteri', 'após', 'apresent', 'ás', 'assim', 'ata', 'bem',
                #   'boa',  'brasilând', 'coloc', 'confer', 'cóp', 'corrob', 'cri', 'cs', 'curs',
                #   'cury', 'currícul', 'dá', 'dar', 'debat', 'deliber', 'deu', 'dire', 'diss', 'diz', 'dua', 'flux', 'dat', 'feit',  'hav', 'nest',
                #  'págin', 'períod', 'tod', 'envi', 'set', 'tem', 'ped', 'justific', 'inform', 'últ', 'text',
                #  'iníci', 'conform', 'vis', 'retom', 'próx', 'outr', 'pod', 'fic', 'grav', 'dev', 'explic', 'prim', 'entr', 'parec',
                #  'segund', 'encaminh', 'poi', 'terceir', 'ser', 'ok', 'particip', 'compartilh', 'dand', 'tard', 'sit', 'fot',
                #    'papel','quórum', 'vez', 'vai', 'junt', 's', 'faustinon', 'onézi', 'vii', 'pres',
                #   'presiddaniel', 'solicit', 'trat', 'hor', 'retorn', 'fal', 'fez',
                #  'moment',  'metrô', 'quatr', 'eleit', 'dess', 'mesm', 'nom', 'segment', 'sei', 'sequ', 'eletrôn', 'discut','lig',
                #  'ness', 'public', 'paul', 'tremembé', 'math', 'event', 'leu', 'lembr', 'ele', 'convit',
                #  'parágraf', 'ofíci', 'concord', 'suger', 'norte', 'atual', 'semestr', 'far', 'faz', 'traz', 'paran', 'badu',
                #  'titul', 'total', 'dispos', 'distrit', 'fim', 'fórum', 'prev', 'previst', 'dentr', 'tribun',
                #  'trê', 'lanç']#,
  remove_list = ['½']
  tokens_stems = [stemmer.stem(word) for word in tokens]
#'½'
  for l in range(len(remove_list)):
      while(remove_list[l] in tokens_stems):
        tokens_stems.remove(remove_list[l])
  # m = 0
  # while(m < (len(tokens_stems)-1)):
  #    if (tokens_stems[m]==tokens_stems[m+1]):
  #        tokens_stems.pop(m+1)
  #    m+=1
  #tokens_stems = [i for i in tokens_stems if len(i)>1]
  return tokens_stems

In [ ]:
def meu_tokenizador(doc, stop_words=nltk.corpus.stopwords.words('portuguese'), stemmer=RSLPStemmer()):
  #return remove_stopwords(doc,stop_words)
  tokens = remove_stopwords(doc,stop_words)
  return stemming(tokens,stemmer)

### Sklearn
* Construção do Modelo Espaço-Vetorial
* Medidas de Similaridade

### Networkx e Plotly


In [ ]:
import plotly.graph_objects as go
import networkx as nx
from networkx.algorithms import community

### Dataset

## Baixar planilhas e descompactá-las

In [ ]:
!gdown --id 1PtOlRCVhtCDL4t_dPqAl8IZJtTvp6dWI
#!gdown --id 1Oon_EcLbmL3MOqw2qptGVPkASix8KRE0
!unzip IQCAE_testeIA.zip
#!unzip IQCAER_Felipe.zip
#!gdown --id 1IeGQqHdKprS184hjxOrAfck4AOLNn3yv  #IQCAE_R_24novCompleto.zip
#!gdown 1A2KegtBoH79UUd_FIV8J4o2jPltrhUet  #IQCAE_R_24novCompleto.zip
#!unzip IQCAE_R_corrigido.zip
#!unzip IQCAE_R_24novCompleto.zip
#!gdown 1TDTROPvpYD8mik2y4JjAId2JAsiP8f8w  #IQCAE_R_24novCompleto.zip
#https://drive.google.com/file/d/1KyQYag8NDBxfdSuaYHkfW3TzObkiGdol/view?usp=share_link
!gdown 1KyQYag8NDBxfdSuaYHkfW3TzObkiGdol
!unzip IQCAE_R_corrigido_27_mar.zip

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1PtOlRCVhtCDL4t_dPqAl8IZJtTvp6dWI
To: /content/IQCAE_testeIA.zip
100% 84.8k/84.8k [00:00<00:00, 91.6MB/s]
Archive:  IQCAE_testeIA.zip
  inflating: IQCAE_testeIA.xlsx      
Downloading...
From: https://drive.google.com/uc?id=1KyQYag8NDBxfdSuaYHkfW3TzObkiGdol
To: /content/IQCAE_R_corrigido_27_mar.zip
100% 214k/214k [00:00<00:00, 88.8MB/s]
Archive:  IQCAE_R_corrigido_27_mar.zip
  inflating: IQCAE_R_corrigido_27_mar.xlsx  


Concatena as planilhas

In [ ]:
import glob
df = pd.ExcelFile('/content/IQCAE_R_corrigido_27_mar.xlsx')
df2 = pd.ExcelFile('/content/IQCAE_testeIA.xlsx')
#!rm /content/IQCAER_Felipe.xlsx
#!rm /content/IQCAE_R_corrigido.xlsx
!rm /content/IQCAE_R_corrigido_27_mar.xlsx
!rm /content/IQCAE_testeIA.xlsx
d = df.parse('Cálculo cardápios', skiprows=2)
ca = df.parse('Dimensões')
arq = []
idx = {}
#idx[0] = [4,21]
#idx[1] = [22, 39]
#idx[2] = [40, 57]
#idx[3] = [58, 75]

idx[0] = [3,21]
idx[1] = [21, 39]
idx[2] = [39, 57]
idx[3] = [57, 75]
idx[4] = [75, 93]

local = d.iloc[:, [1,2]] ## municipio e estado
regiao = {}
regiao['AM'] = ' Norte'
regiao['AP'] = ' Norte'
regiao['AC'] = ' Norte'
regiao['RO'] = ' Norte'
regiao['RR'] = ' Norte'
regiao['TO'] = ' Norte'
regiao['PA'] = ' Norte'
regiao['BA'] = ' Nordeste'
regiao['MA'] = ' Nordeste'
regiao['CE'] = ' Nordeste'
regiao['PB'] = ' Nordeste'
regiao['PE'] = ' Nordeste'
regiao['PI'] = ' Nordeste'
regiao['RN'] = ' Nordeste'
regiao['SE'] = ' Nordeste'
regiao['AL'] = ' Nordeste'
regiao['GO'] = ' Centroeste'
regiao['MT'] = ' Centroeste'
regiao['MS'] = ' Centroeste'
regiao['DF'] = ' Centroeste'
regiao['ES'] = ' Sudeste'
regiao['MG'] = ' Sudeste'
regiao['RJ'] = ' Sudeste'
regiao['SP'] = ' Sudeste'
regiao['PR'] = ' Sul'
regiao['SC'] = ' Sul'
regiao['RS'] = ' Sul'
id = d.iloc[:, [0]]
for i in idx:
  h = d.iloc[:, idx[i][0]:idx[i][1]]
  h.columns = ['horario','tipo', 'descricao', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11','12','13', '14','score']
  h['MUN'] = local['MUN']
  h['ID'] = id['ID']
  h['EST'] = local['EST']

  arq.append(h)
#arq[0]
cardapio = pd.concat(arq)
cardapio = cardapio[[ 'MUN', 'EST','horario','tipo', 'descricao', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11','12','13', '14', 'score', 'ID']]
#cardapio.to_csv('cardapio.csv', sep = ';'
#cardapio['ingredientes'] = cardapio['ingredientes'].fillna('0')
#df1 = d[d.isna().any(axis=1)]
#cardapio=cardapio.dropna(subset=['descricao'])
cardapio=cardapio.dropna()
estado = cardapio['EST'].values.tolist()
index = cardapio['ID'].values.tolist()
reg = [regiao[i] for i in estado]
#reg = [regiao[i] for i in estado]
#text = (cardapio['tipo']+cardapio['horario'] ).values.tolist()
text = (cardapio['descricao']).values.tolist()
tipo = cardapio['tipo'].values.tolist()
horario = cardapio['horario'].values.tolist()
tipo = [i.lower() for i in tipo]
val = {'almoço':1, 'lanche':0}
for i in tipo:
  if i not in val.keys():
    print ("erro:", i)
target2 = [val[i] for i in tipo]
target2 = np.array(target2)
import re
#text = [re.sub("[\(\[].*?[\)\]]", "", x) for x in text]
text = [i.replace('(', ' ') for i in text]
text = [i.replace(')', '') for i in text]
text = [i.replace('moida', 'moída') for i in text]
text = [i.replace('maça', 'maçã') for i in text]
text = [i.replace('Maça', 'Maçã') for i in text]
text = [i.replace('\n', ' ') for i in text]
text = [i.replace('+', ' ').lower() for i in text]
#text = [i+j for i, j in zip(text, reg)]
target = cardapio.iloc[:, 5:20]
#target = cardapio.iloc[:, [18, 19]]
#print(text)
target = target.values.tolist()
target
import numpy as np
target = np.array(target)
#target[:,0] = target2
print(target.shape, target2.shape)
#df1.to_excel('/content/Erro.xls')
#df1
name_list = ca.columns.values.tolist()
print(name_list, type(name_list))
componentes_de_avaliacao = {}
ca
print(text)
for k in name_list:
  componentes_de_avaliacao[k] = ca[k].dropna().tolist()
  #print(componentes_de_avaliacao[k])

(2125, 15) (2125,)
['VARIÁVEIS', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 'Unnamed: 14', 'Unnamed: 15', 14, 'Unnamed: 17'] <class 'list'>
['suco de polpa de cupuaçu, leite em pó, biscoito salgado', 'macarronada, carne moída, suco, salada', 'mingau de fubá, biscoito salgado', 'leite, achocolatado, biscoito doce de maisena, biscoito salgado com margarina', 'pirarucu guisado com abóbora, baião de dois, farinha de mandioca, suco de polpa de caju, melancia', 'pirarucu guisado com abóbora, baião de dois, farinha de mandioca, suco de polpa de caju, melancia', 'pirarucu guisado com abóbora, baião de dois, farinha de mandioca, suco de polpa de caju, melancia', 'vitaminada de banana com aveia, pão', 'melancia e tangerina', 'arroz, beterraba, feijão preto com calabresa', 'chocolate em pó com leite, cuscuz de milho com margarina ', 'baião de três  charque, arroz e feijão da colônia', 'caldo de mandioca, couve e frango', 'arroz, cenoura e salsinha', 'sopa de legumes  batata, cenoura e abóbora, fe

In [ ]:
from sklearn import linear_model, svm
from sklearn.metrics import mean_squared_error
from sklearn.multioutput import MultiOutputClassifier
# Create linear regression object
regr = linear_model.LinearRegression()


In [ ]:
fulltext = [[i,j, k, l, m] for i, j, k, l, m in zip(text, horario, reg, estado, target2)]
fulltext = np.array(fulltext)
#target_final = [[i,j] for i,j in zip(target, target2)]
#t = [i[0] for i in target_final]
#t = np.array(t)
#print(fulltext[:,0])

In [ ]:
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
strt = StratifiedShuffleSplit(n_splits=1, test_size=0.2,random_state=100)
#X_train, y_train, X_test, y_test = iterative_train_test_split(fulltext, target, test_size = 0.2)
X_train, X_test, y_train, y_test = train_test_split(fulltext, target, train_size = 0.80, random_state=5)
X_train_est = np.array([i[3] for i in X_train])
X_train_reg = np.array([i[2] for i in X_train])
X_train_hr = np.array([i[1] for i in X_train])
X_train_text = np.array([i[0] for i in X_train])
X_test_est = np.array([i[3] for i in X_test])
X_test_reg = np.array([i[2] for i in X_test])
X_test_hr = np.array([i[1] for i in X_test])
X_test_text = np.array([i[0] for i in X_test])
ngram = (1,3)
y_train1 = np.array([i for i in y_train])
print(y_train1.shape)
#y_train3 = y_train1[:, y_train1.shape[1]-2]
y_train1 = y_train1[:, :y_train1.shape[1]-1]
y_train2 = np.array([int(i[4]) for i in X_train])

y_test1 = np.array([i for i in y_test])
#y_test3 = y_test1[:, y_test1.shape[1]-2]
y_test1 = y_test1[:, :y_test1.shape[1]-1]
y_test2 = np.array([int(i[4]) for i in X_test])


#y_train2 = y_train2[:, :y_train2.shape[1]]

#y_train1 = y_train
#y_train2 = y_train[:, y_train.shape[1]-1]
#minimo = np.min(y_train2)
#y_train2 = y_train2 - minimo
#y_train2 = y_train2/25
# Train the model using the training sets
print(X_train_text)
#print(y_train2)
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
#VSM = TfidfVectorizer(tokenizer=meu_tokenizador, ngram_range = ngram)#,max_df = 0.95,min_df=0.01)
VSM = CountVectorizer(tokenizer=meu_tokenizador,ngram_range=ngram)#, min_df=2)#, max_features=1000)#, max_df=0.95)#, max_features=500)

VSM2 = CountVectorizer(tokenizer=meu_tokenizador,ngram_range=ngram)#, min_df=2)#, max_df=0.95)#, max_features=500)
VSM3 = CountVectorizer(tokenizer=meu_tokenizador,ngram_range=ngram)
VSM4 = CountVectorizer(tokenizer=meu_tokenizador,ngram_range=ngram)
#VSM.fit(np.concatenate((X_train_hr, X_train_text, X_train_est, X_train_reg)))
VSM.fit(X_train_text)
VSM2.fit(X_train_hr)
VSM3.fit(X_train_reg)
VSM4.fit(X_train_est)
X4 = VSM4.transform(X_train_est)
X3 = VSM3.transform(X_train_reg)
X2 = VSM2.transform(X_train_hr)
X = VSM.transform(X_train_text)
#VSM.vocabulary_
print(y_train1, y_train2)

(1700, 15)
['carreteiro, feijão preto cozido, salada'
 'arroz, carne moída com batata e tomate, salada de pepino, banana'
 'feijão carioca, laranja, arroz branco, isca de carne com legumes' ...
 'banana e pera, bolacha integral salgada, nozes'
 'frango ao molho, arroz, feijão, farofa colorida, salada de repolho'
 'feijão tropeiro e laranja']


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[[1 1 1 ... 0 0 0]
 [1 0 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]] [1 1 1 ... 0 1 1]


In [ ]:

#VSM = CountVectorizer(tokenizer=meu_tokenizador, vocabulary = ['arroz', 'denúnc',
#'problem', 'feij', 'banan', 'frut', 'legum', 'verd', 'ovo'], ngram_range=(1,2))
X = X.toarray()
X2 = X2.toarray()
X3 = X3.toarray()
X4 = X4.toarray()
#print(X.shape)
#print(type(X), type(X2), type(y_train1), type(y_train2))
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [0.1,1, 10, 100], 'gamma': [1,0.1,0.01,0.001],'kernel': ['rbf']}
grid = GridSearchCV(svm.SVC(),param_grid,refit=True,verbose=2)
#regr.fit(y_train1, y_train2)
#RC = linear_model.RidgeClassifier()
#RC.fit(X, y_train1)
MLabel = MultiOutputClassifier(grid)
MLabel.fit(X, y_train1[:, :y_train1.shape[1]-3])
svm_model3 = GridSearchCV(svm.SVC(),param_grid,refit=True,verbose=2)
svm_model4 = GridSearchCV(svm.SVC(),param_grid,refit=True,verbose=2)
svm_model5 = GridSearchCV(svm.SVC(),param_grid,refit=True,verbose=2)
#svm_model3 = svm.SVC()
#svm_model4 = svm.SVC()
#svm_model5 = svm.SVC()

svm_model3.fit(np.concatenate((X, X3), axis=1), y_train1[:, y_train1.shape[1]-3])
svm_model4.fit(np.concatenate((X, X4), axis=1), y_train1[:, y_train1.shape[1]-2])
svm_model5.fit(np.concatenate((X, X2), axis=1), y_train1[:,y_train1.shape[1]-1 ])

#cnn_model = cnn(input_size = X.shape[1], nb_filter = 64, lr = 1e-3, epochs = 500, batch_size = 16, vocab_size=500)
mlp_model= mlp(input_size = X.shape[1], input_size2 = X2.shape[1], input_size3 = X3.shape[1], input_size4 = X4.shape[1], nb_filter = 64, lr = 1e-3, epochs = 50, batch_size = 16)
mlp_model.compile()
mlp_model.summary()
mlp_model.fit([X, X2, X3, X4], [y_train1, y_train2])

X_test1 = VSM.transform(X_test_text)
X_test1 = X_test1.toarray()
#y_tr = mlp_model.predict(X)
X_test2 = VSM2.transform(X_test_hr).toarray()
X_test4 = VSM4.transform(X_test_est).toarray()
X_test3 = VSM3.transform(X_test_reg).toarray()
#regr.fit(y_train1, y_train2)
#y_pred3 = RC.predict(X_test1)
[y_pred, y_pred_tipo] = mlp_model.predict([X_test1, X_test2, X_test3, X_test4])
y_pred2 = MLabel.predict(X_test1)
y_pred3 = svm_model3.predict(np.concatenate((X_test1, X_test3), axis=1))
y_pred4 = svm_model4.predict(np.concatenate((X_test1, X_test4), axis=1))
y_pred5 = svm_model5.predict(np.concatenate((X_test1, X_test2), axis=1))
#y_pred_reg = regr.predict(y_pred)
#y_t2 = y_test[:, y_test.shape[1]-1]-minimo
#y_t2 = y_t2/25
#print("Mean squared error: %.2f" % mean_squared_error(y_t2, y_pred_reg))
#print(y_t2)

#print(np.round(y_pred_reg, 2))
X_latent = mlp_model.get_output([X, X2, X3, X4])
X_test_latent = mlp_model.get_output([X_test1, X_test2, X_test3, X_test4])
MLabel2 = MultiOutputClassifier(svm.SVC())
MLabel2.fit(X_latent[0], y_train1[:, :y_train1.shape[1]-3])
y_pred6 = MLabel2.predict(X_test_latent[0])
svm_model6 = svm.SVC()
svm_model7 = svm.SVC()
svm_model8 = svm.SVC()
svm_model6.fit(X_latent[1], y_train1[:, y_train1.shape[1]-3])
svm_model7.fit(X_latent[2], y_train1[:, y_train1.shape[1]-2])
svm_model8.fit(X_latent[3], y_train1[:,y_train1.shape[1]-1 ])
y_pred7 = svm_model6.predict(X_test_latent[1])
y_pred8 = svm_model7.predict(X_test_latent[2])
y_pred9 = svm_model8.predict(X_test_latent[3])


Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=  54.7s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=  25.2s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=  24.0s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=  22.1s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=  24.4s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=  19.8s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=  16.3s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=  16.0s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=  17.7s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=  17.2s
[CV] END ......................C=0.1, gamma=0.01, kernel=rbf; total time=  12.5s
[CV] END ......................C=0.1, gamma=0.01

In [ ]:
print(y_pred3.shape, y_test1.shape)
y_pred[y_pred<0.5] = 0
y_pred[y_pred>=0.5] = 1
#y_pred_hr[y_pred_hr<0.5] = 0
#y_pred_hr[y_pred_hr>=0.5] = 1
y_pred_tipo[y_pred_tipo<0.5] = 0
y_pred_tipo[y_pred_tipo>=0.5] = 1
y_pred2[y_pred2<0.5] = 0
y_pred2[y_pred2>=0.5] = 1
y_pred3[y_pred3<0.5] = 0
y_pred3[y_pred3>=0.5] = 1
y_pred4[y_pred4<0.5] = 0
y_pred4[y_pred4>=0.5] = 1
y_pred5[y_pred5<0.5] = 0
y_pred5[y_pred5>=0.5] = 1
y_pred6[y_pred6<0.5] = 0
y_pred6[y_pred6>=0.5] = 1
y_pred7[y_pred7<0.5] = 0
y_pred7[y_pred7>=0.5] = 1
y_pred8[y_pred8<0.5] = 0
y_pred8[y_pred8>=0.5] = 1
y_pred9[y_pred9<0.5] = 0
y_pred9[y_pred9>=0.5] = 1
print("Acurácia MLP ", accuracy_score(y_test1[:,:y_test1.shape[1]], y_pred))
#print("Acurácia SVM ", accuracy_score(y_test1[:,:y_test1.shape[1]], y_pred2))
#print("kappa MLP ", cohen_kappa_score(y_test[:,:y_test.shape[1]-1], y_pred))
#print("kappa SVM ", cohen_kappa_score(y_test[:,:y_test.shape[1]-1], y_pred2))
#print("Acurácia Ridge ", accuracy_score(y_test[:,:y_test.shape[1]-1], y_pred3))
#print(confusion_matrix                           (y_test[:,:y_test.shape[1]-1], y_pred))
#y = y_test[:,:y_test.shape[1]-1].flatten()
#y_p = y_pred.flatten()
#print("======Resultado individual=====")
#print("flattened = ", accuracy_score(y, y_p))
#print('MLP')
MLP = []
for i in range(y_test1.shape[1]):
  print("feature ", i,"= ", accuracy_score(y_test1[:,i], y_pred[:,i]))
  MLP.append(accuracy_score(y_test1[:,i], y_pred[:,i]))
  #print("confusion matrix ", i)
  #print(confusion_matrix(y_test[:,i], y_pred[:,i]))
print(y_pred_tipo.shape)
print(accuracy_score(y_test2, y_pred_tipo))
print("Cohen Kappa Score")
for i in range(y_test1.shape[1]):
  print("feature ", i,"= ", cohen_kappa_score(y_test1[:,i], y_pred[:,i]))
print("confusion matrix")
#for i in range(y_test1.shape[1]):
#  print(confusion_matrix(y_test1[:,i], y_pred[:,i]))
#print(accuracy_score(y_test3, y_pred_hr))
print('SVM')
SVM = []
for i in range(y_test1.shape[1]-3):
  print("feature ", i,"= ", accuracy_score(y_test1[:,i], y_pred2[:,i]))
  SVM.append(accuracy_score(y_test1[:,i], y_pred2[:,i]))
  #print("confusion matrix ", i)
  #print(confusion_matrix(y_test1[:,i], y_pred[:,i]))
print("feature 11 ", accuracy_score(y_test1[:,y_test1.shape[1]-3], y_pred3))
print("feature 12 ", accuracy_score(y_test1[:,y_test1.shape[1]-2], y_pred4))
print("feature 13 ", accuracy_score(y_test1[:,y_test1.shape[1]-1], y_pred5))
SVM2 = []
print('cohen kappa svm')
for i in range(y_test1.shape[1]-3):
  print("feature ", i,"= ", cohen_kappa_score(y_test1[:,i], y_pred2[:,i]))
  SVM2.append(cohen_kappa_score(y_test1[:,i], y_pred2[:,i]))
print("feature 11 ", cohen_kappa_score(y_test1[:,y_test1.shape[1]-3], y_pred3))
print("feature 12 ", cohen_kappa_score(y_test1[:,y_test1.shape[1]-2], y_pred4))
print("feature 13 ", cohen_kappa_score(y_test1[:,y_test1.shape[1]-1], y_pred5))
#print('Ridge')
RIDGE = []
#for i in range(y_test.shape[1]-1):
#  print("feature ", i,"= ", accuracy_score(y_test[:,i], y_pred3[:,i]))
#  RIDGE.append(accuracy_score(y_test[:,i], y_pred3[:,i]))
  #print("confusion matrix ", i)
  #print(confusion_matrix(y_test[:,i], y_pred3[:,i]))
print("SVM_MLP")
for i in range(y_test1.shape[1]-3):
  print("feature ", i,"= ", accuracy_score(y_test1[:,i], y_pred6[:,i]))
print("feature  11 ", accuracy_score(y_test1[:,y_test1.shape[1]-3], y_pred7))
print("feature  12 ", accuracy_score(y_test1[:,y_test1.shape[1]-2], y_pred8))
print("feature  13 ", accuracy_score(y_test1[:,y_test1.shape[1]-1], y_pred9))


for i in range(y_test1.shape[1]-3):
  print("feature ", i,"= ", cohen_kappa_score(y_test1[:,i], y_pred6[:,i]))
print("feature  11 ", cohen_kappa_score(y_test1[:,y_test1.shape[1]-3], y_pred7))
print("feature  12 ", cohen_kappa_score(y_test1[:,y_test1.shape[1]-2], y_pred8))
print("feature  13 ", cohen_kappa_score(y_test1[:,y_test1.shape[1]-1], y_pred9))



print("======Resultado individual kappa=====")
#print("flattened = ", accuracy_score(y, y_p))
#print('MLP')
MLP2 = []
#for i in range(y_test1.shape[1]-1):
  #print("feature ", i,"= ", cohen_kappa_score(y_test[:,i], y_pred[:,i]))
  #MLP2.append(cohen_kappa_score(y_test1[:,i], y_pred[:,i]))
  #print("confusion matrix ", i)
  #print(confusion_matrix(y_test[:,i], y_pred[:,i]))
#print('SVM')
SVM2 = []
#for i in range(y_test1.shape[1]-1):
  #print("feature ", i,"= ", cohen_kappa_score(y_test[:,i], y_pred2[:,i]))
  #SVM2.append(cohen_kappa_score(y_test1[:,i], y_pred2[:,i]))
  #print("confusion matrix ", i)
  #print(confusion_matrix(y_test[:,i], y_pred2[:,i]))
#print('Ridge')
#RIDGE2 = []
#for i in range(y_test.shape[1]-1):
#  print("feature ", i,"= ", cohen_kappa_score(y_test[:,i], y_pred3[:,i]))
#  RIDGE2.append(cohen_kappa_score(y_test[:,i], y_pred3[:,i]))
  #print("confusion matrix ", i)
  #print(confusion_matrix(y_test[:,i], y_pred3[:,i]))

(425,) (425, 14)
Acurácia MLP  0.5152941176470588
feature  0 =  0.9435294117647058
feature  1 =  0.9741176470588235
feature  2 =  0.9317647058823529
feature  3 =  0.9011764705882352
feature  4 =  0.9788235294117648
feature  5 =  0.971764705882353
feature  6 =  0.9341176470588235
feature  7 =  0.8917647058823529
feature  8 =  0.9788235294117648
feature  9 =  0.9835294117647059
feature  10 =  0.9647058823529412
feature  11 =  0.8682352941176471
feature  12 =  0.9647058823529412
feature  13 =  0.9811764705882353
(425, 1)
0.9576470588235294
Cohen Kappa Score
feature  0 =  0.8015950204240421
feature  1 =  0.9474512448715787
feature  2 =  0.8494509387176763
feature  3 =  0.801838406714181
feature  4 =  0.9523322906670987
feature  5 =  0.8835829072315559
feature  6 =  0.7659047094464335
feature  7 =  0.6844331095041323
feature  8 =  0.9567381100491998
feature  9 =  0.7501889327399446
feature  10 =  0.8398683780864585
feature  11 =  0.5463210064811285
feature  12 =  0.6632507527336116
feature 

Teste de MC Nemar

In [ ]:
print("teste de mc nemar")
yn = 0
ny = 0
for i in range(y_test1.shape[1]-1):
  yn+= len(y_test1[(np.where((y_test1[:,i]!=y_pred[:,i]) & (y_test1[:,i]==y_pred2[:,i])))])
  ny+= len(y_test1[(np.where((y_test1[:,i]==y_pred[:,i]) & (y_test1[:,i]!=y_pred2[:,i])))])

print("MLP and SVM X = ",((yn-ny)*(yn-ny))/(yn+ny+0.00001))
# yn = 0
# ny = 0
# for i in range(y_test.shape[1]-1):
#   yn+= len(y_test[(np.where((y_test[:,i]!=y_pred[:,i]) & (y_test[:,i]==y_pred3[:,i])))])
#   ny+= len(y_test[(np.where((y_test[:,i]==y_pred[:,i]) & (y_test[:,i]!=y_pred3[:,i])))])

# print("MLP and Ridge X = ",((yn-ny)*(yn-ny))/(yn+ny+0.00001))

# yn = 0
# ny = 0
# for i in range(y_test.shape[1]-1):
#   yn+= len(y_test[(np.where((y_test[:,i]!=y_pred2[:,i]) & (y_test[:,i]==y_pred3[:,i])))])
#   ny+= len(y_test[(np.where((y_test[:,i]==y_pred2[:,i]) & (y_test[:,i]!=y_pred3[:,i])))])

# print("SVM and Ridge X = ",((yn-ny)*(yn-ny))/(yn+ny+0.00001))

teste de mc nemar


IndexError: ignored

### Visualizando a projeção dos vetores resultantes TSNE

In [ ]:
# from sklearn.manifold import TSNE
# tsne = TSNE(n_components=2, verbose=1, perplexity=10, n_iter=300)
# tsne_results = tsne.fit_transform(y_pred)
# print(tsne_results[:,0], tsne_results[:,1])
# import matplotlib.pyplot as plt

In [ ]:
# plt.scatter(tsne_results[:,0], tsne_results[:,1], np.round(y_pred_reg, 2), cmap='viridis')                # plot the points in the array a


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
labels = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13']
x = np.arange(len(labels))  # the label locations
width = 0.2  # the width of the bars
print(len(SVM), SVM)
print(len(MLP), MLP)
fig, ax = plt.subplots()

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Acurácia')
ax.set_xlabel('Componente de avaliação')
ax.legend()
rects1 = ax.bar(x - width/2, SVM, width, label='SVM')
#rects2 = ax.bar(x, RIDGE, width, label='RIDGE')
rects3 = ax.bar(x + width/2, MLP[0:len(MLP)-1], width, label='MLP')
plt.xticks(x, labels)
plt.ylim([0.8, 1.05])
plt.legend(['SVM', 'MLP'], loc='upper right')


plt.show()

In [ ]:

fig, ax = plt.subplots()

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Kappa')
ax.set_xlabel('Componente de avaliação')
ax.legend()
rects1 = ax.bar(x - width/2, SVM2, width, label='SVM')
#rects2 = ax.bar(x, RIDGE2, width, label='RIDGE')
rects3 = ax.bar(x + width/2, MLP2, width, label='MLP')
plt.xticks(x, labels)
plt.ylim([0.2, 1.05])
plt.legend(['SVM', 'MLP'], loc='upper right')


plt.show()

In [ ]:
result = ['Ausência :', ' Presença: ']
#classe = ['tipo de refeição predita: ',
classe = ['Grupo de Cereais e tuberculos ', 'Grupo Leguminosas ', 'Grupo Legumes e Verduras ',
          'Grupo Frutas inteiras ','Grupo ovos, aves e peixes ', 'Grupo leites e derivados ',
          'Doces ou preparacoes doces ', 'ultraprocessados ',  'grupo carnes vermelhas ',
          'preparacoes regionais doces ','alimentos processados ','Alimentos regionais ',
          'Alimentos sociobiodiversidade ', 'Horario compatível com a refeicao ']
print('Erros:')
for i in range(y_test1.shape[1]-1):
  for j in range(y_test1[:,i].shape[0]):

      if(y_test1[j,i]!=y_pred[j,i]):
        print(classe[i], result[int(y_pred[j,i])], X_test[j])
